# **1. Business Understanding / Бизнес-анализ: описание кейса**

К нам обратился владелец небольшой сети кофеен, который планирует расширить 
свой бизнес и открыть несколько новых точек в течение ближайших 4 месяцев.
<br> В связи с этим он хотел бы понять, как распределяются продажи по существующей 
сети кофеен, какие виды продукции вносят наибольший вклад в объем продаж, а 
от каких, возможно, стоит отказаться, кто является основным клиентом сети.
<br> Заказчиком был предоставлен датасет с выгрузкой о транзакциях, товарах, клиентах и торговых точках из его CRM-системы.

**Наша задача** — проанализировать распределение продаж по сети, продажи 
в разрезе категорий продукции, целевую аудиторию кофеен, а также выявить 
возможные взаимосвязи продаж с имеющимися признаками. Полученные результаты необходимо представить в аналитическом отчете, содержащим выводы и рекомендации для Заказчика, которые помогут ему для решения его бизнес-задач.

Исходные данные, представленные в датасете:
1. идентификатор транзакции (transaction_id)
2. количество единиц товара (quantity)
3. цена за единицу товара (unit_price)
4. информация о товаре (product_info), в т.ч. идентификатор товара (product_id), 
   наименование товара (product_name), категория товара (product_category)
5. информация о кофейне (store_info), в т.ч. идентификатор кофейни 
   (sales_outlet_id), город расположения кофейни (store_city)
6.  информация о покупателях (customer_info), в т.ч. идентификатор покупателя 
    (customer_id), дата рождения (birthdate), пол (gender)

**Формат файла** по результатам предобработки: *.csv*

Основные пункты проработанного дизайна исследования представлены в **карточке проекта**.

# **2. Data Understanding / Изучение данных**

## 2.1. Импорт библиотек

In [1]:
# из библиотеки google.colab импортируем класс files при необходимости подгрузки файла с персонального компьютера
# from google.colab import files
# импортируем pandas - основную библиотеку аналитика данных
import pandas as pd
# импортируем библиотеку numpy для возможных расчетов
import numpy as np
# импортируем модуль json для распаковки структур данных формата json
import json
# импортируем статистические библиотеки
import scipy
import scipy.stats as stats
# импортируем модуль pyplot из библиотеки matplotlib для визуализации
from matplotlib import pyplot as plt
# импортируем библиотеку seaborn для визуализации
import seaborn as sns
# для регулярных выражений
import re as re

In [2]:
# установить размер графиков
sns.set(rc={'figure.figsize':(8,4)})

## 2.2. Подгрузка файла

Предварительно скачиваем файл в ту же папку, где расположен блокнот, чтобы можно было работать в стационарной среде

## 2.3. Чтение файла

In [3]:
df = pd.read_csv('coffee_shop_hard.csv', sep = ',')

## 2.4. Обзор датасета

In [4]:
df.head()

,transaction_id,quantity,unit_price,product_info,store_info,customer_info
0,0,1,162.50,"{""product_id"":52,""product_name"":""Traditional B...","{""sales_outlet_id"":3,""store_city"":""Санкт-Петер...","{""customer_id"":558,""birthdate"":""1983-02-25"",""g..."
1,1,2,227.50,"{""product_id"":27,""product_name"":""Brazilian Lg""...","{""sales_outlet_id"":3,""store_city"":""Санкт-Петер...","{""customer_id"":781,""birthdate"":""1991-07-29"",""g..."
2,2,2,162.50,"{""product_id"":46,""product_name"":""Serenity Gree...","{""sales_outlet_id"":3,""store_city"":""Санкт-Петер...","{""customer_id"":788,""birthdate"":""1995-02-23"",""g..."
3,3,2,162.50,"{""product_id"":23,""product_name"":""Our Old Time ...","{""sales_outlet_id"":3,""store_city"":""Санкт-Петер...","{""customer_id"":683,""birthdate"":""1999-02-06"",""g..."
4,4,1,159.25,"{""product_id"":34,""product_name"":""Jamaican Coff...","{""sales_outlet_id"":3,""store_city"":""Санкт-Петер...","{""customer_id"":99,""birthdate"":""1967-01-29"",""ge..."


In [5]:
df.shape

(24909, 6)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24909 entries, 0 to 24908
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   transaction_id  24909 non-null  int64  
 1   quantity        24909 non-null  int64  
 2   unit_price      24909 non-null  float64
 3   product_info    24909 non-null  object 
 4   store_info      24909 non-null  object 
 5   customer_info   24909 non-null  object 
dtypes: float64(1), int64(2), object(3)
memory usage: 1.1+ MB


## 2.5. Выявление проблем

1. Наличие признаков, не представляющих ценности в рамках кейса (будет понятно после распаковки вложенных значений в признаках 'product_info', 'store_info', 'customer_info' )
2. Возможно наличие полных дубликатов
3. Наличие незаполненных значений (в исходных признаках незаполненных значений нет; после распаковки вложенных структур в признаках 'product_info', 'store_info', 'customer_info' проведем соответствующую проверку)
4. Наличие вложенных значений (словари в признаках 'product_info', 'store_info', 'customer_info', изначально данные в формате json-строки)
5. Приведение признаков в необходимый формат (три первые столбца датасета: 'transaction_id', 'quantity' и 'unit_price' уже находятся в необходимых нам форматах; после распаковки вложенных значений в признаках 'product_info', 'store_info', 'customer_info' необходимо будет привести вновь полученные признаки в необходимый формат)
6. Необходимость расчета признака 'age' (от признака 'birthdate'), признака 'total' (общая сумма чека)
7. Возможно введение дополнительных новых признаков, необходимых для проведения анализа (кроме признаков, полученных в результате распаковки вложенных структур)
8. Наличие закодированных значений (в исходном датасете нет значений, необходимых в декодировании)
9. Возможные другие проблемы, которые могут быть выявлены в процессе предобработки данных

# **3. Data Preparation / Предобработка данных**

## 3.1. Проверка и исключение полных дубликатов

In [7]:
df_copy = df.copy()  # создадим копию исходного датафрейма

In [8]:
print(df_copy.duplicated().sum()) # кол-во полных дубликатов в датафрейме
print(df_copy.duplicated(subset = 'transaction_id').sum()) # совпадает с  кол-вом дубликатов 
# по столбцу transaction_id

57
57


In [9]:
df_copy['transaction_id'].nunique() # кол-во уникальных транзакций

24852

In [10]:
# исключаем полные дубликаты из датафрейма
df_copy.drop_duplicates(inplace=True)
df_copy.shape # теперь в датасете только уникальные транзакции

(24852, 6)

In [11]:
# делаем перезапись индексного столбца
df_copy.reset_index(drop=True, inplace=True)
df_copy.index # проверем диапазон индексов

RangeIndex(start=0, stop=24852, step=1)

## 3.2. Распаковка вложенных структур данных

In [12]:
df_copy.head(3)

,transaction_id,quantity,unit_price,product_info,store_info,customer_info
0,0,1,162.5,"{""product_id"":52,""product_name"":""Traditional B...","{""sales_outlet_id"":3,""store_city"":""Санкт-Петер...","{""customer_id"":558,""birthdate"":""1983-02-25"",""g..."
1,1,2,227.5,"{""product_id"":27,""product_name"":""Brazilian Lg""...","{""sales_outlet_id"":3,""store_city"":""Санкт-Петер...","{""customer_id"":781,""birthdate"":""1991-07-29"",""g..."
2,2,2,162.5,"{""product_id"":46,""product_name"":""Serenity Gree...","{""sales_outlet_id"":3,""store_city"":""Санкт-Петер...","{""customer_id"":788,""birthdate"":""1995-02-23"",""g..."


**Необходимо обработать:** 'product_info', 'store_info', 'customer_info'

### 'product_info'

In [13]:
# обзор случайного значения по признаку
df_copy['product_info'][229]

'{"product_id":29,"product_name":"Columbian Medium Roast Rg","product_category":"Coffee"}'

Необходимо записать значения по ключам 'product_id', 'product_name' и 'product_category' в отдельные столбцы, данные не содержат пустые значения, поэтому проверку на пропуски можно опустить

Напишем функцию, которая будет приводить json-строку в формат словаря и извлекать из него значение 
по заданному ключу

In [14]:
def prod_id_func (data):
    result = json.loads(data)['product_id']
    return result

def prod_name_func (data):
    result = json.loads(data)['product_name']
    return result

def prod_category_func (data):
    result = json.loads(data)['product_category']
    return result

In [15]:
# применяем функции к столбцу
df_copy['product_id'] = df_copy['product_info'].apply(prod_id_func)
df_copy['product_name'] = df_copy['product_info'].apply(prod_name_func)
df_copy['product_category'] = df_copy['product_info'].apply(prod_category_func)

In [16]:
# контроль результата
df_copy[['product_id', 'product_name', 'product_category']]

,product_id,product_name,product_category
0,52,Traditional Blend Chai Rg,Tea
1,27,Brazilian Lg,Coffee
2,46,Serenity Green Tea Rg,Tea
3,23,Our Old Time Diner Blend Rg,Coffee
4,34,Jamaican Coffee River Sm,Coffee
...,...,...,...
24847,41,Cappuccino Lg,Coffee
24848,87,Ouro Brasileiro shot,Coffee
24849,25,Brazilian Sm,Coffee
24850,44,Peppermint Rg,Tea


### 'store_info'

In [17]:
# обзор случайного значения по признаку
df_copy['store_info'][209]

'{"sales_outlet_id":3,"store_city":"Санкт-Петербург"}'

Необходимо записать значения по ключам 'sales_outlet_id' и 'store_city' в отдельные столбцы, данные не содержат пустые значения, поэтому проверку на пропуски можно опустить

Воспользуемся ранее написанной функцией, которая будет приводить json-строку в формат словаря и извлекать из него значение 
по заданному ключу

In [18]:
def outlet_id_func (data):
    result = json.loads(data)['sales_outlet_id']
    return result

def city_func (data):
    result = json.loads(data)['store_city']
    return result

In [19]:
# применяем функции к столбцу
df_copy['sales_outlet_id'] = df_copy['store_info'].apply(outlet_id_func)
df_copy['store_city'] = df_copy['store_info'].apply(city_func)

In [20]:
# контроль результата
df_copy[['sales_outlet_id', 'store_city']]

,sales_outlet_id,store_city
0,3,Санкт-Петербург
1,3,Санкт-Петербург
2,3,Санкт-Петербург
3,3,Санкт-Петербург
4,3,Санкт-Петербург
...,...,...
24847,8,Moscow
24848,8,Moscow
24849,8,Moscow
24850,8,Moscow


### 'customer_info'

In [21]:
# обзор случайного значения по признаку
df_copy['customer_info'][543]

'{"customer_id":395,"birthdate":"1962-05-11","gender":"женский"}'

Необходимо записать значения по ключам 'customer_id', 'birthdate' и 'gender' в отдельные столбцы, данные не содержат пустые значения, поэтому проверку на пропуски можно опустить

Воспользуемся ранее написанной функцией, которая будет приводить json-строку в формат словаря и извлекать из него значение по заданному ключу

In [22]:
def customer_id_func (data):
    result = json.loads(data)['customer_id']
    return result

def birthdate_func (data):
    result = json.loads(data)['birthdate']
    return result

def gender_func (data):
    result = json.loads(data)['gender']
    return result

In [23]:
# применяем функции к столбцу
df_copy['customer_id'] = df_copy['customer_info'].apply(customer_id_func)
df_copy['birthdate'] = df_copy['customer_info'].apply(birthdate_func)
df_copy['gender'] = df_copy['customer_info'].apply(gender_func)

In [24]:
# контроль результата
df_copy[['customer_id', 'birthdate', 'gender']]

,customer_id,birthdate,gender
0,558,1983-02-25,женский
1,781,1991-07-29,не указан
2,788,1995-02-23,не указан
3,683,1999-02-06,женский
4,99,1967-01-29,мужской
...,...,...,...
24847,8026,1962-10-25,мужской
24848,8401,1954-11-05,мужской
24849,8412,1953-09-16,мужской
24850,8030,1994-01-10,женский


## 3.3. Исключение признаков, не представляющих ценности для дальнейшего выполнения поставленной задачи

In [25]:
df_copy.head()

,transaction_id,quantity,unit_price,product_info,store_info,customer_info,product_id,product_name,product_category,sales_outlet_id,store_city,customer_id,birthdate,gender
0,0,1,162.50,"{""product_id"":52,""product_name"":""Traditional B...","{""sales_outlet_id"":3,""store_city"":""Санкт-Петер...","{""customer_id"":558,""birthdate"":""1983-02-25"",""g...",52,Traditional Blend Chai Rg,Tea,3,Санкт-Петербург,558,1983-02-25,женский
1,1,2,227.50,"{""product_id"":27,""product_name"":""Brazilian Lg""...","{""sales_outlet_id"":3,""store_city"":""Санкт-Петер...","{""customer_id"":781,""birthdate"":""1991-07-29"",""g...",27,Brazilian Lg,Coffee,3,Санкт-Петербург,781,1991-07-29,не указан
2,2,2,162.50,"{""product_id"":46,""product_name"":""Serenity Gree...","{""sales_outlet_id"":3,""store_city"":""Санкт-Петер...","{""customer_id"":788,""birthdate"":""1995-02-23"",""g...",46,Serenity Green Tea Rg,Tea,3,Санкт-Петербург,788,1995-02-23,не указан
3,3,2,162.50,"{""product_id"":23,""product_name"":""Our Old Time ...","{""sales_outlet_id"":3,""store_city"":""Санкт-Петер...","{""customer_id"":683,""birthdate"":""1999-02-06"",""g...",23,Our Old Time Diner Blend Rg,Coffee,3,Санкт-Петербург,683,1999-02-06,женский
4,4,1,159.25,"{""product_id"":34,""product_name"":""Jamaican Coff...","{""sales_outlet_id"":3,""store_city"":""Санкт-Петер...","{""customer_id"":99,""birthdate"":""1967-01-29"",""ge...",34,Jamaican Coffee River Sm,Coffee,3,Санкт-Петербург,99,1967-01-29,мужской


In [26]:
# создание списка "ненужных" колонок - нераспакованные "словари"
columns = ['product_info', 'store_info', 'customer_info']

In [27]:
df_copy.drop(columns=columns, inplace=True)

In [28]:
df_copy.info() 
# проверяем список оставшихся столбцов, в датафрейме нет пустых значений 
# проверяем индексацию и смотрим типы данных по столбцам 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24852 entries, 0 to 24851
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   transaction_id    24852 non-null  int64  
 1   quantity          24852 non-null  int64  
 2   unit_price        24852 non-null  float64
 3   product_id        24852 non-null  int64  
 4   product_name      24852 non-null  object 
 5   product_category  24852 non-null  object 
 6   sales_outlet_id   24852 non-null  int64  
 7   store_city        24852 non-null  object 
 8   customer_id       24852 non-null  int64  
 9   birthdate         24852 non-null  object 
 10  gender            24852 non-null  object 
dtypes: float64(1), int64(5), object(5)
memory usage: 2.1+ MB


In [29]:
df_copy.head() # визуальный контроль результата

,transaction_id,quantity,unit_price,product_id,product_name,product_category,sales_outlet_id,store_city,customer_id,birthdate,gender
0,0,1,162.50,52,Traditional Blend Chai Rg,Tea,3,Санкт-Петербург,558,1983-02-25,женский
1,1,2,227.50,27,Brazilian Lg,Coffee,3,Санкт-Петербург,781,1991-07-29,не указан
2,2,2,162.50,46,Serenity Green Tea Rg,Tea,3,Санкт-Петербург,788,1995-02-23,не указан
3,3,2,162.50,23,Our Old Time Diner Blend Rg,Coffee,3,Санкт-Петербург,683,1999-02-06,женский
4,4,1,159.25,34,Jamaican Coffee River Sm,Coffee,3,Санкт-Петербург,99,1967-01-29,мужской


## 3.4. Преобразование данных в необходимый формат

**Необходимо преобразовать** следующие признаки:
1. 'birthdate' - в формат даты и времени datetime
2. Остальные признаки в необходимом нам формате

In [30]:
# установим нужный формат данных с помощью метода to_datetime()
# для корректного перевода значений укажем формат в параметре format,
# где на первом месте стоит год, а далее месяц и день (как в исходном датасете)
df_copy.birthdate = pd.to_datetime(df_copy.birthdate, format="%Y-%m-%d")
df_copy.birthdate 

0       1983-02-25
1       1991-07-29
2       1995-02-23
3       1999-02-06
4       1967-01-29
           ...    
24847   1962-10-25
24848   1954-11-05
24849   1953-09-16
24850   1994-01-10
24851   1994-01-10
Name: birthdate, Length: 24852, dtype: datetime64[ns]

## 3.5. Форматирование текстовых данных

**Необходимо проверить и отформатировать** следующие признаки:
1. 'product_name'
2. 'product_category'
3. 'store_city'
4. 'gender'

### 'product_name'

In [31]:
# проверим список уникальных значений с помощью метода unique()
# при этом сначала выведем количество уникальных значений, используя функцию len(),
# а сам список отсортируем по алфавиту с помощью функции sorted() для удобства проверки
print(len(df_copy.product_name.unique()))
sorted(df_copy.product_name.unique())

80


['Almond Croissant',
 'Brazilian - Organic',
 'Brazilian Lg',
 'Brazilian Rg',
 'Brazilian Sm',
 'Cappuccino',
 'Cappuccino Lg',
 'Carmel syrup',
 'Chili Mayan',
 'Chocolate Chip Biscotti',
 'Chocolate Croissant',
 'Chocolate syrup',
 'Civet Cat',
 'Columbian Medium Roast',
 'Columbian Medium Roast Lg',
 'Columbian Medium Roast Rg',
 'Columbian Medium Roast Sm',
 'Cranberry Scone',
 'Croissant',
 'Dark chocolate',
 'Dark chocolate Lg',
 'Dark chocolate Rg',
 'Earl Grey',
 'Earl Grey Lg',
 'Earl Grey Rg',
 'English Breakfast',
 'English Breakfast Lg',
 'English Breakfast Rg',
 'Espresso Roast',
 'Espresso shot',
 'Ethiopia',
 'Ethiopia Lg',
 'Ethiopia Rg',
 'Ethiopia Sm',
 'Ginger Biscotti',
 'Ginger Scone',
 'Guatemalan Sustainably Grown',
 'Hazelnut Biscotti',
 'Hazelnut syrup',
 'I Need My Bean! Diner mug',
 'I Need My Bean! Latte cup',
 'I Need My Bean! T-shirt',
 'Jamacian Coffee River',
 'Jamaican Coffee River Lg',
 'Jamaican Coffee River Rg',
 'Jamaican Coffee River Sm',
 'Jumbo 

В датасете 80 уникальных значений по признаку product_name.
Изучив значения, можно выделить **следующие проблемы**:
1. слово "Jamacian" (написано с орфографической ошибкой) приведем в правильный вид "Jamaican" 
2. целесообразно привести все слова к единому регистру для удобства работы и исключения возможных незамеченных ошибок.

In [32]:
# с помощью лямбда-функции и регулярного выражения заменим слово во всем столбце
df_copy.product_name = df_copy.product_name.apply(lambda x: re.sub('Jamacian', 'Jamaican', x))

# приведем значения к единому регистру (строчным буквам) с помощью метода str.lower()
df_copy.product_name = df_copy.product_name.str.lower()

In [33]:
# контроль результата
print(f'кол-во уникальных значений в product_name - {df_copy.product_name.nunique()}')
print(f'кол-во уникальных значений в product_id - {df_copy.product_id.nunique()}')
# кол-во уникальных элементов совпадает в двух столбцах (так и должно быть, ошибок нет)
sorted(df_copy.product_name.unique())

кол-во уникальных значений в product_name - 80
кол-во уникальных значений в product_id - 80


['almond croissant',
 'brazilian - organic',
 'brazilian lg',
 'brazilian rg',
 'brazilian sm',
 'cappuccino',
 'cappuccino lg',
 'carmel syrup',
 'chili mayan',
 'chocolate chip biscotti',
 'chocolate croissant',
 'chocolate syrup',
 'civet cat',
 'columbian medium roast',
 'columbian medium roast lg',
 'columbian medium roast rg',
 'columbian medium roast sm',
 'cranberry scone',
 'croissant',
 'dark chocolate',
 'dark chocolate lg',
 'dark chocolate rg',
 'earl grey',
 'earl grey lg',
 'earl grey rg',
 'english breakfast',
 'english breakfast lg',
 'english breakfast rg',
 'espresso roast',
 'espresso shot',
 'ethiopia',
 'ethiopia lg',
 'ethiopia rg',
 'ethiopia sm',
 'ginger biscotti',
 'ginger scone',
 'guatemalan sustainably grown',
 'hazelnut biscotti',
 'hazelnut syrup',
 'i need my bean! diner mug',
 'i need my bean! latte cup',
 'i need my bean! t-shirt',
 'jamaican coffee river',
 'jamaican coffee river lg',
 'jamaican coffee river rg',
 'jamaican coffee river sm',
 'jumbo 

### 'product_category'

In [34]:
# выведем количество уникальных значений, используя функцию nunique(),
# выведем отсортированный список уникальных значений с помощью метода unique()
print(df_copy.product_category.nunique())
sorted(df_copy.product_category.unique())

9


['Bakery',
 'Branded',
 'Coffee',
 'Coffee beans',
 'Drinking Chocolate',
 'Flavours',
 'Loose Tea',
 'Packaged Chocolate',
 'Tea']

В датасете 9 уникальных значений по признаку product_category.
Изучив значения, можно выделить **следующие проблемы**:
1. целесообразно привести все слова к единому регистру для удобства работы.

In [35]:
# приведем значения к единому регистру (строчным буквам) с помощью метода str.lower()
df_copy.product_category = df_copy.product_category.str.lower()

In [36]:
# контроль результата
print(f'кол-во уникальных значений в product_category - {df_copy.product_category.nunique()}')
sorted(df_copy.product_category.unique())

кол-во уникальных значений в product_category - 9


['bakery',
 'branded',
 'coffee',
 'coffee beans',
 'drinking chocolate',
 'flavours',
 'loose tea',
 'packaged chocolate',
 'tea']

### 'store_city'

In [37]:
# выведем количество уникальных значений, используя функцию nunique(),
# выведем отсортированный список уникальных значений с помощью метода unique()
print(df_copy.store_city.nunique())
sorted(df_copy.store_city.unique())

3


['Moscow', 'Москва', 'Санкт-Петербург']

Изучив значения, можно выделить **следующие проблемы**:
1. необходимо заменить 'Moscow' на 'Москва', т.к. в этом столбце по сути всего 2 уникальных значения.
2. целесообразно привести все слова к единому регистру для удобства работы.

In [38]:
# с помощью лямбда-функции и регулярного выражения произведем замену слова во всем столбце
df_copy.store_city = df_copy.store_city.apply(lambda x: re.sub('Moscow', 'Москва', x))

In [39]:
# приведем значения к единому регистру (строчным буквам) с помощью метода str.lower()
df_copy.store_city = df_copy.store_city.str.lower()

In [40]:
# контроль результата
print(df_copy.store_city.nunique())
sorted(df_copy.store_city.unique())

2


['москва', 'санкт-петербург']

### 'gender'

In [41]:
# выведем количество уникальных значений, используя функцию nunique(),
# выведем отсортированный список уникальных значений с помощью метода unique()
print(df_copy.gender.nunique())
sorted(df_copy.gender.unique())

3


['женский', 'мужской', 'не указан']

In [42]:
df_copy[df_copy['gender'] == 'не указан']

,transaction_id,quantity,unit_price,product_id,product_name,product_category,sales_outlet_id,store_city,customer_id,birthdate,gender
1,1,2,227.50,27,brazilian lg,coffee,3,санкт-петербург,781,1991-07-29,не указан
2,2,2,162.50,46,serenity green tea rg,tea,3,санкт-петербург,788,1995-02-23,не указан
24,24,1,243.75,38,latte,coffee,3,санкт-петербург,722,1961-06-18,не указан
39,39,2,243.75,38,latte,coffee,3,санкт-петербург,788,1995-02-23,не указан
79,79,2,159.25,34,jamaican coffee river sm,coffee,3,санкт-петербург,717,1958-11-29,не указан
...,...,...,...,...,...,...,...,...,...,...,...
24825,49816,2,162.50,46,serenity green tea rg,tea,8,москва,8340,2000-04-23,не указан
24833,49844,2,195.00,47,serenity green tea lg,tea,8,москва,8486,2000-04-26,не указан
24837,49860,2,243.75,38,latte,coffee,8,москва,8161,1995-07-04,не указан
24839,49862,2,143.00,25,brazilian sm,coffee,8,москва,8185,1997-07-28,не указан


В датасете 3 уникальных значения по признаку gender. Изучив значения, можно сделать **следующие выводы**:
1. все значения приведены к единому регистру.
2. в датасете 5779 наблюдений, в которых не указан пол. Удалить их мы не можем, т.к. это больше 20% всех наблюдений.

## 3.6. Расчет дополнительных признаков

### 'age'

In [43]:
# для расчета возраста необходимо из текущей даты вычесть дату рождения (признак "birthdate")
# получаем текущую дату с помощью метода Timestamp.today()
current_day = pd.Timestamp.today()
current_day

Timestamp('2024-07-26 13:20:13.279619')

In [44]:
# извлечем значение года из даты с помощью атрибута year
current_year = current_day.year
current_year

2024

In [45]:
# рассчитаем значения возраста и сохраним полученные значения в признак age
# при этом необходимо каждую дату рождения перевести в значение года аналогично через атрибут year
df_copy['age'] = current_year - df_copy.birthdate.apply(lambda x: x.year)

In [46]:
# исключим исходный столбец birthdate, т.к. он больше не участвует в расчетах
df_copy.drop(columns='birthdate', inplace=True)

In [47]:
# визуальный контроль результата
df_copy.head()

,transaction_id,quantity,unit_price,product_id,product_name,product_category,sales_outlet_id,store_city,customer_id,gender,age
0,0,1,162.50,52,traditional blend chai rg,tea,3,санкт-петербург,558,женский,41
1,1,2,227.50,27,brazilian lg,coffee,3,санкт-петербург,781,не указан,33
2,2,2,162.50,46,serenity green tea rg,tea,3,санкт-петербург,788,не указан,29
3,3,2,162.50,23,our old time diner blend rg,coffee,3,санкт-петербург,683,женский,25
4,4,1,159.25,34,jamaican coffee river sm,coffee,3,санкт-петербург,99,мужской,57


### 'total'

In [48]:
# рассчитаем дополнительный признак total (общая сумма по каждой транзакции) 
# по формуле (quantity * unit_price)
df_copy['total'] = df_copy['quantity'] * df_copy['unit_price']

## 3.7. Проверка и сохранение датасета

### Общий датасет

In [49]:
# немного преобразуем наш датафрейм
# расположим столбцы в порядке, удобном для просмотра
columns = ['transaction_id', 'quantity', 'unit_price', 'total', 'product_id', 'product_name', 'product_category', 'sales_outlet_id', 'store_city', 'customer_id', 'gender', 'age']
df_copy = df_copy[columns]
# немного сократим названия некоторых колонок
df_copy = df_copy.rename(columns = {'sales_outlet_id':'outlet_id', 'store_city':'city' })

In [50]:
# обзор размеров датафрейма
df_copy.shape

(24852, 12)

In [51]:
# обзор типов данных, пропущенных значений, диапазона индексов
df_copy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 24852 entries, 0 to 24851
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   transaction_id    24852 non-null  int64  
 1   quantity          24852 non-null  int64  
 2   unit_price        24852 non-null  float64
 3   total             24852 non-null  float64
 4   product_id        24852 non-null  int64  
 5   product_name      24852 non-null  object 
 6   product_category  24852 non-null  object 
 7   outlet_id         24852 non-null  int64  
 8   city              24852 non-null  object 
 9   customer_id       24852 non-null  int64  
 10  gender            24852 non-null  object 
 11  age               24852 non-null  int64  
dtypes: float64(2), int64(6), object(4)
memory usage: 2.3+ MB


In [52]:
# обзор содержания полученного датасета
df_copy.head()

,transaction_id,quantity,unit_price,total,product_id,product_name,product_category,outlet_id,city,customer_id,gender,age
0,0,1,162.50,162.50,52,traditional blend chai rg,tea,3,санкт-петербург,558,женский,41
1,1,2,227.50,455.00,27,brazilian lg,coffee,3,санкт-петербург,781,не указан,33
2,2,2,162.50,325.00,46,serenity green tea rg,tea,3,санкт-петербург,788,не указан,29
3,3,2,162.50,325.00,23,our old time diner blend rg,coffee,3,санкт-петербург,683,женский,25
4,4,1,159.25,159.25,34,jamaican coffee river sm,coffee,3,санкт-петербург,99,мужской,57


In [53]:
# сохраним результаты в файл в формате csv
df_copy.to_csv('coffee_shop_prepared.csv')

## 3.8. Выводы по разделу:

1. Был представлен датасет с выгрузкой о транзакциях, товарах, клиентах и торговых точках из CRM-системы Заказчика. Файл в формате csv, содержащий 24909 строк (объектов) и 6 столбцов (признаков).
2. В ходе очистки данных были исключены:
* признаки, не представляющие ценности в рамках указанных условий (после распаковки вложенных значений в признаках 'product_info', 'store_info', 'customer_info');
* "полные" дубликаты (ошибочно задвоенные записи);
3. Произведена проверка на наличие незаполненных значений (в исходных признаках и в полученных признаках после распаковки вложенных структур в признаках 'product_info', 'store_info', 'customer_info'). Пустых значений не обнаружено. 
4. Были распакованы полуструктурированные данные, представленные в формате json. В результате распаковки нами были получены следующие признаки: 'product_id' (id продукта), 'product_name' (наименование продукта), 'product_category' (категория продукта), 'sales_outlet_id' (id точки продаж), 'store_city' (город), 'customer_id' (id клиента), 'gender' (пол).
5. Значения признаков были преобразованы в соответствующие форматы, необходимые для последующего анализа: признак birthdate в формат datetime.
6. Дополнительно произведен расчет признаков 'age' (возраст) и 'total' (общая сумма по каждой транзакции).
7. Отформатированы и унифицированы значения текстовых признаков 'product_name' (наименование продукта), 'product_category' (категория продукта) и 'store_city' (город).
8. По итогу раздела принято решение о сохранении преобразованного датасета для последующего анализа:
* в разрезе уникальных транзакций (24852 объектов, 12 признаков): файл в формате csv (coffee_shop_prepared.csv).
9. В связи с обнаруженными "полными" дубликатами следует дополнительно оформить пояснительную записку, содержащую обратную связь по исходным данным, и направить ее Заказчику.